In [ ]:
# Module Import

from pandas import read_csv
from pandas import DataFrame
from numpy import random
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from joblib import dump
from matplotlib import pyplot

from xgboost import plot_importance

In [ ]:
!echo $BIGML_USERNAME

In [ ]:
# data Import and Cleanning

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)
df.fillna(0, inplace=True)
df.to_csv('train_full.csv')
train_full= read_csv('train_full.csv')


In [ ]:
# Stabilisation du notebook

SEED = 42
random.seed(SEED)

In [ ]:
# Split de la data

VAL_SIZE = 0.2
train, val = train_test_split(train_full, test_size=VAL_SIZE)

In [ ]:
# sanity check

print("Train full shape: " + str(train_full.shape))
print("Train shape: " + str(train.shape))
print("Val shape: " + str(val.shape))

In [ ]:
# Save the data

train.to_csv("train.csv")
val.to_csv("val.csv")

In [ ]:
# Load des data

train = read_csv('train.csv', index_col=0)
val = read_csv('val.csv', index_col=0)

In [ ]:
# création d'une table "Y" qui contient uniquement la target

target_column = 'SeriousDlqin2yrs'
y_train = train[target_column].values
print(y_train)

In [ ]:
# Creation d'une table "X" qui contient tout sauf la target

X_train = train.drop(target_column, axis=1).values
print(X_train)

In [ ]:
# Idem pour le Val set

X_val = val.drop(target_column, axis=1).values
y_val = val[target_column].values

In [ ]:
# # Creation du modele Classifier
# # this class contains the implementation of the Random Forest learning technique for classification

# model1 = RandomForestClassifier()

# model1.fit(X_train, y_train)

In [ ]:
# Creation et execution du modele XGBC

model2 = XGBClassifier()

model2.fit(X_train, y_train)

In [ ]:
plot_importance(model2, feature_name=)
pyplot.show()

In [ ]:
# # Apply model 1 to val set

# y_val_proba = model1.predict_proba(X_val)
# # print(y_val_proba)

# y_val_proba = y_val_proba[:,1]
# print(y_val_proba)

In [ ]:
# Apply model 2 to val set

y_val_proba = model2.predict_proba(X_val)
# print(y_val_proba)

y_val_proba = y_val_proba[:,1]  
print(y_val_proba)

In [ ]:
# Calcul de l'AUC

from sklearn import metrics
metrics.roc_auc_score(y_val, y_val_proba)

In [ ]:
# on save le modele en Joblib

bestmodel = model2
dump(bestmodel, 'bestmodel.joblib')

In [ ]:
# Compute confusion matrix

threshold = 0.5
y_val_pred = [1 if p >= threshold else 0 for p in y_val_proba]
pprint(y_val_pred)

In [ ]:
metrics.confusion_matrix(y_val, y_val_pred)

## Utilisation du modele sur le Test_full

In [ ]:
# data Import and Cleanning

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv', index_col=0)
df.fillna(0, inplace=True)
df.to_csv('test_full.csv')
test_full= read_csv('test_full.csv')

In [ ]:
# création d'une table "Y" qui contient uniquement la target

target_column = 'SeriousDlqin2yrs'
y_test = test_full[target_column].values
print(y_train)

In [ ]:
# Creation d'une table "X" qui contient tout sauf la target

X_test = test_full.drop(target_column, axis=1).values
pprint(X_train)

In [ ]:
# Apply model 2 to test_full

y_test_proba = model2.predict_proba(X_test)
# print(y_val_proba)

y_test_proba = y_test_proba[:,1]  
print(y_test_proba)

In [ ]:
df_R = DataFrame(data={'Probability' : y_test_proba})
df_R.index += 1
df_R.index.names = ['Id']
df_R.to_csv('kaggle_sklearn.csv')
df_R

In [ ]:
# Soumission Kaggle

submission_file = "Kaggle_sklearn.csv"

In [ ]:
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMesomeCredit")
